In [1]:
import requests
import chess
import chess.pgn
import json
from chess.engine import Cp
import pandas as pd

In [2]:
url = "https://explorer.lichess.ovh/lichess"

params = {
    "variant": "standard",
    "speeds": "blitz,rapid,classical",
    "ratings": "2200,2500",
    "fen": None
}


url_eval = "https://lichess.org/api/cloud-eval"

params_eval = {
    "variant": "standard",
    "multiPv": "1",
    "fen": None
}

In [3]:
game = chess.pgn.Game()

game.headers["Event"] = "Example"

In [4]:
node = game.add_variation(chess.Move.from_uci("g2g4"))

In [5]:
params["fen"] = node.board().fen()

In [6]:
response = requests.get(url, params=params)

In [7]:
tree = json.loads(response.content.decode())

In [8]:
tree['moves'][0]['uci']

'd7d5'

In [9]:
node.ply()

1

In [10]:
def get_cloud_eval(node):
    
    params_eval['fen'] = node.board().fen()
    response = requests.get(url_eval, params=params_eval)
    tree = json.loads(response.content.decode())
        
    return tree['pvs'][0]['cp']

In [11]:
def compute_evaluations(node, tree_move, total_games):
        
    w = tree_move['white']
    b = tree_move['black']
    d = tree_move['draws']

    tot = w+b+d

    tree_move['tot_games'] = tot

    tree_move['white'] = (int)(w/tot * 100)
    tree_move['black'] = (int)(b/tot * 100)
    tree_move['draws'] = (int)(d/tot * 100)
    
    tree_move['strongest_practical'] = tree_move['white'] if (node.board().turn) else tree_move['black']
        
    tree_move['perc'] = (int)((float)(tot)/total_games * 100.)
    
    n = node.add_variation(chess.Move.from_uci(tree_move['uci']))
    tree_move['eval'] = get_cloud_eval(n)/100.
        

In [12]:
def filter_moves(move_list, bIsWhiteToMove, bIsWhiteRepertoire):
    """filtra le mosse per trovare o le candidate per il giocatore oppure quelle da considerare dagli avversari

    Args:
        move_list (list): tree['moves']
        bIsWhiteToMove (bool): deve muovere il bianco ?
        bIsWhiteRepertoire (bool): booleano che indica se si sta costruendo un repertorio per il bianco o per il nero

    Returns:
        list: lista delle mosse candidate e loro relativi commenti
    """
    
    ret_moves = []
    
    bIsPlayerTurn = (bIsWhiteToMove == bIsWhiteRepertoire)
    
    if(bIsPlayerTurn):
        sorted_moves = sorted(move_list, key=lambda x: x["strongest_practical"], reverse=True)
        
        for m in sorted_moves:
            if m["eval_pos"] > 3: # se la mossa non è tra le prime tre del motore la scarto
                continue
            if(bIsWhiteToMove and m['eval'] < -1) or (not bIsWhiteToMove and m['eval'] > 1): # se la mossa ha una valutazione del motore non accettabile (< -1 e tocca la bianco o > 1 e tocca al nero) la scarto
                continue
            ret_moves.append(m)
            return ret_moves # se tocca al giocatore considero solo una mossa alla volta
    else:
        sorted_moves = sorted(move_list, key=lambda x: x["perc"], reverse=True)
        perc_sum = 0
        for m in sorted_moves:
            ret_moves.append(m)
            perc_sum += m["perc"]
            if(perc_sum > 80): # considero mosse fino al punto in cui ho coperto almeno l'80% delle mosse giocate
                return ret_moves
            
    return ret_moves

In [13]:
total_games = tree['white'] + tree['draws'] + tree['black']

for move in tree['moves']:
    compute_evaluations(node, move, total_games)
    
eval_sorted_moves = sorted(tree['moves'], key=lambda x: x["eval"])

toMove_sorted_moves = [] #score relativo da db in base a chi deve muovere

if(node.board().turn):
    toMove_sorted_moves = sorted(tree['moves'], key=lambda x: x["white"], reverse=True)
else:
    toMove_sorted_moves = sorted(tree['moves'], key=lambda x: x["black"], reverse=True)
  
toMove_sorted_moves

for i, item in enumerate(tree['moves']):
    item["freq_pos"] = i
    item["eval_pos"] = eval_sorted_moves.index(item)
    item["dbscore_pos"] = toMove_sorted_moves.index(item)

In [14]:
pd.DataFrame(tree['moves'])

,uci,san,averageRating,white,draws,black,game,tot_games,strongest_practical,perc,eval,freq_pos,eval_pos,dbscore_pos
0,d7d5,d5,2305,47,5,46,None,80205,46,52,-1.28,0,0,8
1,e7e5,e5,2302,45,5,48,None,17545,48,11,-0.93,1,1,2
2,d7d6,d6,2292,46,5,47,None,9888,47,6,-0.60,2,5,4
3,c7c5,c5,2290,46,5,48,None,9053,48,5,-0.88,3,2,3
4,h7h5,h5,2306,44,5,50,None,7955,50,5,-0.44,4,7,0
5,g7g6,g6,2290,46,6,47,None,6770,47,4,-0.35,5,8,5
6,c7c6,c6,2289,46,5,47,None,6612,47,4,-0.47,6,6,6
7,e7e6,e6,2278,46,5,47,None,5516,47,3,-0.61,7,4,7
8,g8f6,Nf6,2293,48,4,46,None,2192,46,1,0.25,8,10,9
9,b8c6,Nc6,2282,45,5,49,None,2103,49,1,-0.81,9,3,1


In [15]:
candidates = filter_moves(tree['moves'], bIsWhiteToMove = False, bIsWhiteRepertoire = False)

In [16]:
pd.DataFrame(candidates)

,uci,san,averageRating,white,draws,black,game,tot_games,strongest_practical,perc,eval,freq_pos,eval_pos,dbscore_pos
0,b8c6,Nc6,2282,45,5,49,None,2103,49,1,-0.81,9,3,1


# TODO: 
* se è il giocatore a muovere bisogna scegliere la mossa candidata con la logica: 
> 1. si ordinando le mosse inversamente allo score del db
> 2. si guarda la prima mossa, la scelgo se è tra le prime 3 (configurabile) del motore + se la valutazione del motore non è peggiore di +/- 1 (se ho il bianco non voglio scegliere linee minori di -1. Viceversa per il nero)

In [17]:
print('\U0001F534', '\U0001F7E0', '\U0001F7E1', '\U0001F7E2')

🔴 🟠 🟡 🟢


In [18]:
from IPython.display import clear_output
import time

for i in range(1, 101):
    time.sleep(0.1)
    clear_output(wait=True)
    print(f"Caricamento: {i}%")


Caricamento: 100%


In [33]:
board = chess.Board()
notation = board.variation_san([node.move.from_uci('g2g4')])


In [34]:
print(f"{board.fullmove_number}.{notation}")


1.1. g4


In [35]:
node.move.san()

AttributeError: 'Move' object has no attribute 'san'

In [36]:
board = chess.Board()
san_move = board.san(node.move)

In [37]:
san_move

'g4'

In [41]:
b = node.board()

In [42]:
b.fullmove_number

1

In [43]:
b.peek()

Move.from_uci('g2g4')

In [44]:
turn = b.turn

In [45]:
turn

False